In [1]:
from typing import TypedDict, Optional
from langgraph.graph import StateGraph, START, END

from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv(override=True)
my_api_key = os.getenv("OPENAI_API_KEY")
# print (f'Key is {my_api_key}')


client = OpenAI(api_key=my_api_key)


# --- Shared State ---
class LibraryState(TypedDict):
    question: Optional[str]
    faq_flag: Optional[bool]
    faq_answer: Optional[str]
    checkout_flag: Optional[bool]
    checkout_answer: Optional[str]
    final_answer: Optional[str]

def ClassifierAgent(state: LibraryState):
    print(f"ClassifierAgent Initial state: {state}")

    question = state["question"]

    messages = [
        {
            "role": "system",
            "content": (
                "You are a classifier agent in a library system.\n"
                "Determine whether the question involves:\n"
                "1. Library FAQs (hours, rules, membership)\n"
                "2. Book availability or checkout\n\n"
                "Return ONLY valid JSON:\n"
                "{\n"
                '  "faq_flag": true | false,\n'
                '  "checkout_flag": true | false\n'
                "}"
            )
        },
        {"role": "user", "content": question}
    ]

    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=messages,
    )

    import json
    parsed = json.loads(response.choices[0].message.content)

    return {
        "faq_flag": bool(parsed.get("faq_flag", False)),
        "checkout_flag": bool(parsed.get("checkout_flag", False)),
    }
def FAQAgent(state: LibraryState):
    print(f"FAQAgent state: {state}")

    if not state.get("faq_flag"):
        return {"faq_answer": None}

    messages = [
        {"role": "system", "content": "You answer library FAQ questions."},
        {"role": "user", "content": state["question"]}
    ]

    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=messages,
    )

    return {
        "faq_answer": response.choices[0].message.content
    }



def CheckoutAgent(state: LibraryState):
    print(f"CheckoutAgent state: {state}")

    if not state.get("checkout_flag"):
        return {"checkout_answer": None}

    messages = [
        {"role": "system", "content": "You answer questions about book availability and checkout."},
        {"role": "user", "content": state["question"]}
    ]

    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=messages,
    )

    return {
        "checkout_answer": response.choices[0].message.content
    }
def ResponseAgent(state: LibraryState):
    print(f"ResponseAgent state: {state}")

    faq = state.get("faq_answer")
    checkout = state.get("checkout_answer")

    parts = []
    if faq:
        parts.append(f"FAQ Information:\n{faq}")
    if checkout:
        parts.append(f"Checkout Information:\n{checkout}")

    combined_context = "\n\n".join(parts)

    messages = [
        {
            "role": "system",
            "content": "Combine the information into a helpful response."
        },
        {
            "role": "user",
            "content": f"Question: {state['question']}\n\n{combined_context}"
        }
    ]

    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=messages,
    )

    return {
        "final_answer": response.choices[0].message.content
    }
# --- Build the Graph ---
builder = StateGraph(LibraryState)
builder.add_node("ClassifierAgent", ClassifierAgent)
builder.add_node("FAQAgent", FAQAgent)
builder.add_node("CheckoutAgent", CheckoutAgent)
builder.add_node("ResponseAgent", ResponseAgent)

builder.add_edge(START, "ClassifierAgent")
builder.add_edge("ClassifierAgent", "FAQAgent")
builder.add_edge("ClassifierAgent", "CheckoutAgent")
builder.add_edge("FAQAgent", "ResponseAgent")
builder.add_edge("CheckoutAgent", "ResponseAgent")
builder.add_edge("ResponseAgent", END)

graph = builder.compile()

In [2]:
result = graph.invoke({"question": "Is The new best seller available? When does library open?"})
print("\n--- Final Answer ---")
print(result["final_answer"])

ClassifierAgent Initial state: {'question': 'Is The new best seller available? When does library open?'}
CheckoutAgent state: {'question': 'Is The new best seller available? When does library open?', 'faq_flag': True, 'checkout_flag': True}FAQAgent state: {'question': 'Is The new best seller available? When does library open?', 'faq_flag': True, 'checkout_flag': True}

ResponseAgent state: {'question': 'Is The new best seller available? When does library open?', 'faq_flag': True, 'faq_answer': 'I can help with both, but I need a couple details:\n\n- Which library or branch are you asking about?\n- Do you have the exact title or author for the “new bestseller”?\n\nIf you want to proceed now:\n- Availability: I can check the library catalog for the title and tell you if a copy is available, on hold, or checked out. If it’s not available, you can usually place a hold to be notified when it’s back on the shelf.\n- Hours: Opening hours vary by branch and day. Tell me the library name or you

In [ ]:
# # --- Visualize ---
# print(graph.get_graph().draw_ascii())
# graph.get_graph().draw_png("images/agentic_ai_library.png")
# print("Graph saved as agentic_ai_library.png")

In [3]:
# --- Run ---
result = graph.invoke({"question": 
                       '''When does Cupertino library open? I would like to check out a book as 
                       well.'''})
print("\n--- Final Answer ---")
print(result["final_answer"])

ClassifierAgent Initial state: {'question': 'When does Cupertino library open? I would like to check out a book as \n                       well.'}
CheckoutAgent state: {'question': 'When does Cupertino library open? I would like to check out a book as \n                       well.', 'faq_flag': True, 'checkout_flag': True}FAQAgent state: {'question': 'When does Cupertino library open? I would like to check out a book as \n                       well.', 'faq_flag': True, 'checkout_flag': True}

ResponseAgent state: {'question': 'When does Cupertino library open? I would like to check out a book as \n                       well.', 'faq_flag': True, 'faq_answer': 'I can help with that.\n\n- Hours: Cupertino Library hours change by day. The quickest way to get today’s opening times is to check the official site or call the library. I can look up the current hours for you if you’d like.\n\n- Checking out a book: Here’s how to get started\n  - Get a library card: If you live in Santa Clara

In [4]:
result = graph.invoke({"question": "Is The Hobbit available? When does the library open"})
print("\n--- Final Answer ---")
print(result["final_answer"])


ClassifierAgent Initial state: {'question': 'Is The Hobbit available? When does the library open'}
CheckoutAgent state: {'question': 'Is The Hobbit available? When does the library open', 'faq_flag': True, 'checkout_flag': True}FAQAgent state: {'question': 'Is The Hobbit available? When does the library open', 'faq_flag': True, 'checkout_flag': True}

ResponseAgent state: {'question': 'Is The Hobbit available? When does the library open', 'faq_flag': True, 'faq_answer': 'I can help, but availability and hours depend on your library branch. Could you tell me:\n\n- Which library or city/branch are you using?\n- Do you want The Hobbit in a physical copy, or an eBook/audiobook?\n- Do you want today’s opening hours or the regular hours for the branch?\n\nIf you’d like, I can guide you to check in your library’s catalog and show you how to place a hold or find digital copies. For a quick check now, you can:\n- Search the library catalog for “The Hobbit” by J.R.R. Tolkien, then filter by form